## Author: Ben Cliff

# About the data  
##### The dataset loaded into this notebook is a .csv file obtained from the CDC's preventative measures site.  
##### The sample subject of this dataset is U.S. citizens who are 65 years or older. The sample is subdivided by gender, State and city.
##### The target or signal of this dataset is the percentage of citizens (broken down by gender and location) who are reported to have taken preventative measures such as immunizations and select cancer screenings against common illnesses.
##### [Bethlehem insert commentary here about what trends we want to see]  

# Purpose of this notebook:
##### Create a clean .csv file to work from for further analysis
##### Perform preliminary analysis as well as diagnostics of the data
##### Create visualizations from this notebook for upload onto our blog site

# Reading and cleaning the dataset

In [1]:
# Importing powerful data manipulation library
import pandas as pd
import numpy as np
import geopandas as gpd
from geopandas import GeoDataFrame
# import shapely
from shapely.geometry import Point

In [2]:
# Reading in prevention data
prevent_df = pd.read_csv('../data/preventativedata.csv')

# Removing with little to no information
prevent_df = prevent_df.drop(columns=['Data_Value_Unit', 'Data_Value_Footnote_Symbol', 'Data_Value_Footnote', 'TractFIPS', 'CategoryID', 'StateDesc', 'Data_Value_Type', 'DataSource',
'DataValueTypeID', 'Category'])

In [3]:
# Removing undescores
prevent_df.columns = prevent_df.columns.str.replace('_', '')
# prevent_df.columns

In [4]:
# Sorting values by unique ID; creating new index and dropping the old one
prevent_df.sort_values(by=['UniqueID'], ascending=True).reset_index().head(3).drop(columns=['index'])

,Year,StateAbbr,CityName,GeographicLevel,UniqueID,Measure,DataValue,LowConfidenceLimit,HighConfidenceLimit,PopulationCount,GeoLocation,MeasureId,CityFIPS,ShortQuestionText
0,2016,HI,Honolulu,City,15003,Older adult men aged >=65 Years who are up to ...,31.4,31.1,31.7,"953,207","(21.4588039305, -157.973296737)",COREM,15003,Core preventive services for older men
1,2016,HI,Honolulu,City,15003,Older adult women aged >=65 Years who are up t...,30.4,30.1,30.7,"953,207","(21.4588039305, -157.973296737)",COREW,15003,Core preventive services for older women
2,2016,AL,Birmingham,City,107000,Older adult men aged >=65 Years who are up to ...,32.9,32.2,33.7,"212,237","(33.5275663773, -86.7988174678)",COREM,107000,Core preventive services for older men


In [5]:
temp_list = []

longs = []

prevent_df['GeoLocation'] = [x.replace('(', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.replace(')', '') for x in prevent_df['GeoLocation']]
prevent_df['GeoLocation'] = [x.split(',') for x in prevent_df['GeoLocation']]
#prevent_df['GeoLocation'] = [float(x) for x in prevent_df['GeoLocation']]


In [6]:
lats = [float(x[0]) for x in prevent_df['GeoLocation']]
longs = [float(x[1]) for x in prevent_df['GeoLocation']]

In [7]:
prevent_df['lats'] = lats
prevent_df['longs'] = longs
# prevent_df.head(5)

# Data Visualization and Analysis

In [8]:
# Importing visualization tool
import plotly.graph_objects as go
import plotly.express as px

In [9]:
prevent_df['PopulationCount'] = [x.replace(',', '') for x in prevent_df['PopulationCount']]
prevent_df['PopulationCount'] = prevent_df['PopulationCount'].astype(int)

In [10]:
# Adding new column to prevent_df: top 5 states for geospatial analysis
temp_list = []
for x in prevent_df['StateAbbr']:
    if x == 'CA' or x == 'TX' or x == 'FL' or x == 'IL' or x == 'MI':
        temp_list.append('yes')
    else:
        temp_list.append('no')

In [11]:
prevent_df['top5state'] = temp_list
# prevent_df.head(10)

In [12]:
# Extracting only the male records of the dataset to get the unique counts of surveys by cities
men_df = prevent_df.loc[prevent_df['MeasureId'] == 'COREM']
women_df = prevent_df.loc[prevent_df['MeasureId'] == 'COREW']

# Isolating the top 5 counts of cities in separate dataframe
top_5_states = men_df.loc[(men_df['StateAbbr'] == 'CA') | (men_df['StateAbbr'] == 'TX') | (men_df['StateAbbr'] == 'FL')
| (men_df['StateAbbr'] == 'IL') | (men_df['StateAbbr'] == 'MI')]

# Isolating all other 45 states in separate dataframe
other_states_df = men_df.loc[(men_df['StateAbbr'] != 'CA') & (men_df['StateAbbr'] != 'TX') & (men_df['StateAbbr'] != 'FL')
& (men_df['StateAbbr'] != 'IL') & (men_df['StateAbbr'] != 'MI')]

In [13]:
# Creating separate data for the histogram counts
x = other_states_df['StateAbbr']
y = top_5_states['StateAbbr']

# Creating graph object
fig = go.Figure()
fig.add_trace(go.Histogram(histfunc='count', x=x, name='State Participants'))
fig.add_trace(go.Histogram(histfunc='count', x=y, name='Top 5 States represented', marker_color='#330C73'))
fig.update_layout(title_text='Cities Count of Survey', xaxis_title_text='State Abbreviation', yaxis_title_text='Count')
fig.show()

In [14]:

geometry = [Point(xy) for xy in zip(lats, longs)]
gdf = GeoDataFrame(prevent_df, geometry=geometry)

world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
usa = world.loc[world['name'] == 'United States of America']

fig = px.scatter_geo(men_df, lat=men_df['lats'], lon=men_df['longs'], width=800, height=400, color=men_df['top5state'])
fig.update_layout(title = 'PLACES survey locations', geo_scope='usa', showlegend=False)
fig.show()

## Working Hypothesis
- Direction for analysis: Geospatial -> why are certain states recieving more attention than others?
- Can we make comparison of the female and male participants of this study -> are more women taking the CDC preventative measures? Are they not? Are they the same?

In [15]:
# Cross tabulation of the Data by State and the Population Count of each survey location
new_df = pd.crosstab(index=[men_df['StateAbbr'], pd.cut(men_df['PopulationCount'], [0, 50000, 100000, 250000, 500000, 1000000])], columns=men_df['PopulationCount'], margins=True, margins_name='Counts')

In [16]:
# Find a state's location site's population by state abbreviation
new_df.loc['TX'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,18
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,20
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,3
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,3


In [17]:
new_df.loc['CA'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,0,55
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,53
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,1,0,2


In [18]:
new_df.loc['FL'].sort_values('PopulationCount')

PopulationCount,42417,51400,59466,66135,66154,66194,66455,66588,66702,66748,...,731424,741206,787033,790390,805235,820445,821784,945942,953207,Counts
PopulationCount,,,,,,,,,,,,,,,,,,,,,
"(50000, 100000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
"(100000, 250000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
"(250000, 500000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,2
"(500000, 1000000]",0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,1


In [19]:
agg_men = men_df.groupby('StateAbbr')
agg_women = women_df.groupby('StateAbbr')

In [20]:
state_counts = men_df.groupby('StateAbbr').count().Year

In [21]:
state_counts.describe()

count     51.000000
mean       9.803922
std       17.838183
min        1.000000
25%        2.000000
50%        6.000000
75%       10.000000
max      121.000000
Name: Year, dtype: float64

# Hypothesis Testing

In [58]:
# Plot data for men, then women to see if t-test is appropriate for testing hypothesis
fig = px.histogram(prevent_df, 'DataValue', color='MeasureId', marginal='box', barmode='overlay', width=1000, height=500)
fig.update_layout(title='Overlaid Histogram of Prevention Rates by Sex', xaxis_title_text = 'Prevention Rates (%)')
fig.show()

The histogram above is more informative than expected prior to its creation. Ultimately, it shows that the data is normally distributed enough for a student t-test to test the null hypothesis that the male and female subgroups have identical average values. In this case, the averages of the rates that the groups succeed in satisfying the CDC's preventative measures requirements. Furthermore, the data shows that the male participants generally meet the CDC preventative measures guidelines than the female participants.

COREW - Signifies females  
COREM - Signifies males

In [57]:
# Grouped data by gender, then state
# Taken arithmetic mean of the DataValue column
# Looking at DataValue for man and women by state 

mean_for_men = agg_men.mean()['DataValue']
mean_for_women = agg_women.mean()['DataValue']

In [31]:
# Importing statistics functions from scipy for the hypothesis testing
from scipy.stats import ttest_ind
from scipy.special import logsumexp

In [33]:
# Creating list objects out of series objects
men_list = list(mean_for_men)
women_list = list(mean_for_women)

In [36]:
# Creating function that iterates over each item of a list and inserts the log sum of exponential of item
# Function returns the newly generated list
def logsums(means):
    a_list = []
    for x in range(len(means)):
        a_list.append(logsumexp(means[x]))
    return a_list

In [37]:
logs_men = logsums(mean_for_men)
logs_women = logsums(mean_for_women)

In [39]:
# Performing student t-test against the aggregated female and male groups' prevention rates
t_stat, p = ttest_ind(logs_men, logs_women)
print(f't={t_stat}, p={p}')

t=5.021515947041427, p=2.2408081609896807e-06


The p-value for this student t-test < 0.05. Therefore, we fail to reject the null hypothesis that the averages of the rates of men and women satisfying the CDC's preventative measures guidelines are identical.